## Exporting a MobileNetV2 model to ONNX
You can export any Fireball model to ONNX using the [exportToOnnx](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToOnnx) function. This notebook shows how to use this function to create an ONNX model. It assumes that a trained MobileNetV2 model already exists in the ```Models``` directory. Please refer to the notebook [Image Classification with MobileNetV2](MobileNetV2.ipynb) for more info about using a pretrained MobileNetV2 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)
- [Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)
- [Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)

Note: Fireball uses the [onnx](https://github.com/onnx/onnx) python package to export models to ONNX. We also use the [onnxruntime](https://onnxruntime.ai) here to run and evaluate the onnx models.

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='upto4'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)

orgFileName = "Models/MobileNetV2RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/MobileNetV2RRPRQR.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      435        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      279        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      265        
S2_L4_BN         112 112 16 

## Export the model and check the exported ONNX model

In [2]:
from fireball.datasets.imagenet import ImageNetDSet

onnxFileName = orgFileName.replace(".fbm",".onnx")

model.exportToOnnx(onnxFileName, runQuantized=True, classNames=ImageNetDSet.classNames, 
                   modelDocStr="Fireball example: MobileNetV2 Model")

# Check the exported model. This throws exceptions if something is wrong with the exported model.
import onnx
from onnx import shape_inference

onnxModel = onnx.load(onnxFileName)
onnx.checker.check_model(onnxModel)


Exporting to ONNX model "Models/MobileNetV2RRPRQR.onnx" ... 
    Processed all 27 layers.                                     
    Saving to "Models/MobileNetV2RRPRQR.onnx" ... Done.
Done (57.92 Sec.)


## Using netron to visualize the exported model
We can now visualize the model's network structure using the [netron](https://github.com/lutzroeder/netron) package.

In [3]:
import netron
import platform

if platform.system() == 'Darwin':      # Running on MAC
    netron.start(onnxFileName)   
else:
    import socket
    hostIp = socket.gethostbyname(socket.gethostname())
    netron.start(onnxFileName, address=(hostIp,8084))

Serving 'Models/MobileNetV2RRPRQR.onnx' at http://10.1.16.58:8084


## Running inference on the exported model
To verify the exported model, we can now run inference on it. Here we load an image and do the required pre-processing before passing it to the exported model as input. We then print the top-3 most probable predicted labels for the image.

In [4]:
import cv2
import numpy as np
imageFileName = 'CoffeeMug.jpg'

img = cv2.imread(imageFileName)     # Reads image in BGR order
img = np.float32(img)[..., ::-1]    # Convert to RGB

# Resize the image to 256x256
imgSize = img.shape[:2]
ratio = 256.0/min(imgSize)
newSize = (int(np.round(imgSize[1]*ratio)), int(np.round(imgSize[0]*ratio)))

# Note: INTER_AREA is best when shrinking and CV_INTER_CUBIC is best when enlarging
img = cv2.resize(img, newSize,  interpolation = (cv2.INTER_AREA if ratio<1.0 else cv2.INTER_CUBIC))

# Now crop the center 224x224 image
dw = newSize[0] - 224
dh = newSize[1] - 224
resizedImg = img[dh//2:dh//2+224, dw//2:dw//2+224,:]

# Normalize the image values to the range: -1 .. 1
inputImage = ((np.float32(resizedImg)/127.5)-1.0)
inputImage = np.transpose(inputImage, (2,0,1))    # Onnx expects channel-first images

# Inference using the ONNX model and "onnxruntime"
import onnxruntime as ort
options = ort.SessionOptions()
options.intra_op_num_threads = 4
session = ort.InferenceSession(onnxModel.SerializeToString(), options, providers=['CPUExecutionProvider'])
    
print('input: %s, ouput: %s'%(session.get_inputs()[0].name,session.get_outputs()[0].name))
y = session.run(['ClassProbs'],{'InputImage':[inputImage]})

classProbs = y[0][0]
top3Indexes = np.argsort(classProbs)[-3:][::-1]    # Indexes of classes with 3 highest probs (decreasing order)
top3Porbs = classProbs[top3Indexes]
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(ImageNetDSet.classNames[top3Indexes[i]], top3Porbs[i])) 

input: InputImage, ouput: ClassProbs
Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.611419)
    cup (0.335758)
    pitcher (0.014754)


## Also look at

[Exporting MobileNetV2 Model to CoreML](MobileNetV2-CoreML.ipynb)

[Exporting MobileNetV2 Model to TensorFlow](MobileNetV2-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with MobileNetV2](MobileNetV2.ipynb)

[Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)

[Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)

[Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)

